# Практическое задание курса Light Auto ML. Часть 3 - Custom Solution

<details>
<summary>Описание задания</summary>

Основная задача - выбрать и решить соревнование с платформы Kaggle.com  (http://kaggle.com/) , используя два подхода:
1. Подготовить базовое решение (бейзлайн) с помощью Light Auto ML (LAMA)
2. Реализовать альтернативное решение без использования LAMA

Требования к выбору соревнования
- Можно выбрать как текущие, так и прошедшие денежные соревнования
- Другие типы соревнований необходимо согласовать с куратором курса
- Нельзя использовать простые соревнования типа Titanic

Цели проекта
- Превзойти результаты бейзлайна на LAMA
- Продемонстрировать качественный код
- Использовать стандартные подходы к организации кода (например, Pipeline)
- Провести качественный EDA
- Предоставить подробное описание и обоснование гипотез

Критерии оценки
1. Анализ целевой переменной (максимум 1 балл)
[0.5] Численный анализ:
Для регрессии: распределение таргета, поиск аномальных значений
Для классификации: распределение количества классов
[0.5] Визуализация статистик:
- Изолированный анализ
- Анализ во временном контексте

2. Анализ признаков (максимум 4 балла)
[0.5] Типизация признаков (числовые, категориальные, временные) и их распределения
[0.5] Выявление аномальных значений
[0.5] Анализ зависимостей между признаками
[0.5] Анализ пропущенных значений
[0.5] Определение важности признаков (корреляции с таргетом)
[1.0] Графическая визуализация минимум 3-х пунктов выше
[0.5] Анализ возможных преобразований и генерации новых признаков

3. Моделирование (максимум 3.5 балла)
[0.25] Обоснование стратегии разделения данных (train-test split)
Особое внимание уделить предотвращению утечки данных
[0.25] LAMA бейзлайн:
- Минимум 2 различные конфигурации
- Выбор лучшего решения
[3.0] Собственное решение (если не удалось побить LLama baseline: 3 x 1.0 балл за различные пайплайны/попытки):
- Выбор модели
- Построение пайплайна (препроцессинг, обработка пропусков, генерация признаков, отбор признаков, финальная модель/ансамбль)
- Оптимизация гиперпараметров

4. Общие требования к коду (максимум 1.5 балла)
[0.5] Чистый код:
- Оформление ноутбука
- Соответствие PEP 8
- Правильное именование переменных и функций
- Документирование функций
[0.5] Качество кода:
- Следование принципам SOLID
- Отсутствие спагетти-кода
- Обработка предупреждений и ошибок
- Логгирование
[0.5] Структура решения:
- Оформление в виде self-contained pipeline
- Использование стандартных инструментов (например, sklearn pipeline)

Итоговая оценка
Максимальный балл: 10
9-10 баллов: оценка 5А
7-8.5 баллов: оценка 4В
5-6.5 баллов: оценка 3D
Менее 5 баллов: требуется пересдача

Ожидания
Работа должна представлять собой мини-исследование с:
1) Проработкой и проверкой гипотез
2) Оценкой результатов
3) Обоснованием выбора пайплайна
4) Документированием процесса исследования

</details>

### Часть 3 - Custom Solution

In [ ]:
# импорт нужных библиотек
import pandas as pd

from sklearn.model_selection import GroupKFold
from sklearn.metrics import mean_absolute_error
from lightgbm import LGBMRegressor
from lightgbm.callback import early_stopping
import optuna
import numpy as np

In [ ]:
# загрузка данных
train = pd.read_csv('./data/train.csv')
test  = pd.read_csv('./data/test.csv')
sample = pd.read_csv('./data/sample_submission.csv')

print(f"Размер тренировочной выборки: {train.shape}")
print(f"Размер тестовой выборки: {test.shape}")
print(f"Размер sample выборки: {sample.shape}")

Размер тренировочной выборки: (6036000, 8)
Размер тестовой выборки: (4024000, 7)
Размер sample выборки: (4024000, 2)


##### **3.1 Генерации новых признаков**

Вспомним наш стек добавочных признаков:

**1. Lag-признаки**
- `u_in_lag_1/2/3` - предыдущие значения управляющего сигнала
- так как корреляция внутри циклов в 2-3 раза сильнее глобальной

**2. Difference признаки**
- `u_in_diff` - скорость изменения управляющего сигнала
- динамика изменений критична для временных моделей

**3. Cumulative sum**
- `u_in_cumsum` - накопленный объем воздуха
-  физический смысл - интеграл потока

**4. Взаимодействия R×C**
- `R_x_C` - произведение параметров легких
- график показал нелинейное влияние комбинаций

**5. Позиция в цикле**
- `time_position` - порядковый номер шага внутри breath_id (0-79)
- позволяет модели определять фазу дыхания независимо от абсолютного времени

In [7]:
def create_features(df):
    """
    Создание признаков на основе анализа:
    - Lag features
    - Difference
    - Cumulative sum
    - R×C interaction
    """
    df = df.copy()
    
    for lag in [1, 2, 3]:
        df[f'u_in_lag_{lag}'] = df.groupby('breath_id')['u_in'].shift(lag)

    df['u_in_diff'] = df.groupby('breath_id')['u_in'].diff()
    df['u_in_cumsum'] = df.groupby('breath_id')['u_in'].cumsum()
    df['R_x_C'] = df['R'] * df['C']
    df['time_position'] = df.groupby('breath_id').cumcount()
    
    # Заполнение NaN в lag-признаках нулями
    lag_cols = [f'u_in_lag_{i}' for i in [1, 2, 3]] + ['u_in_diff']
    df[lag_cols] = df[lag_cols].fillna(0)
    
    print(f"Created {len(lag_cols) + 3} new features")
    return df

train_fe = create_features(train)

# Список всех признаков
ORIGINAL_FEATURES = ['R', 'C', 'time_step', 'u_in', 'u_out']
NEW_FEATURES = ['u_in_lag_1', 'u_in_lag_2', 'u_in_lag_3', 
                'u_in_diff', 'u_in_cumsum', 'R_x_C', 'time_position']
ALL_FEATURES = ORIGINAL_FEATURES + NEW_FEATURES

print(f"Всего признаков: {len(ALL_FEATURES)}")
print(f"   Оригинальные: {len(ORIGINAL_FEATURES)}")
print(f"   Новые: {len(NEW_FEATURES)}")
print(f"\nНовые признаки: {NEW_FEATURES}")

Created 7 new features
Всего признаков: 12
   Оригинальные: 5
   Новые: 7

Новые признаки: ['u_in_lag_1', 'u_in_lag_2', 'u_in_lag_3', 'u_in_diff', 'u_in_cumsum', 'R_x_C', 'time_position']


In [ ]:
train_fe = create_features(train)
test_fe  = create_features(test)

FEATURES = ['R','C','time_step','u_in','u_out','u_in_lag_1','u_in_lag_2','u_in_lag_3',
            'u_in_diff','u_in_cumsum','R_x_C','time_position']
TARGET = 'pressure'

Created 7 new features
Created 7 new features


### **3.2 Выбор модели и стратегия валидации**

**Почему выбор упарл на LightGBM?**
1. **Результаты LAMA**: LightGBM показал лучшие результаты в LAMA экспериментах
2. **Скорость обучения**: быстрее CatBoost, что критично для Optuna оптимизации и эффективно работает с большими данными (более 6М+ строк)
3. **Встроенная обработка категориальных признаков**: R, C, u_out

**Стратегия валидации: GroupKFold**
- **5 фолдов** по breath_id - целые циклы дыхания не пересекаются между train/val
- **Предотвращение утечки**: модель не видит другие шаги того же цикла во время валидации
- **Метрика**: MAE

**Early Stopping**
- Остановка обучения через 100 итераций без улучшения
- Защита от переобучения
- Ускорение Optuna (не обучаем плохие конфигурации до конца)

**Оптимизируемые параметры:**
- `learning_rate`: 0.01 - 0.1 (log scale) - скорость обучения
- `num_leaves`: 31 - 128 - сложность деревьев
- `subsample`: 0.6 - 1.0 - доля строк для обучения каждого дерева
- `colsample_bytree`: 0.6 - 1.0 - доля признаков для каждого дерева

**Фиксированные параметры:**
- `n_estimators`: 4000 - максимальное число деревьев (с early stopping)
- `random_state`: 42 - воспроизводимость

**Настройки Optuna:**
- 15 trials - компромисс между качеством и временем
- 1800 секунд timeout - максимум 30 минут
- Минимизация OOF MAE

In [ ]:
# Optuna
def objective(trial):
    params = {
        'n_estimators': 4000,
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.1),
        'num_leaves': trial.suggest_int('num_leaves', 31, 128),
        'subsample': trial.suggest_uniform('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.6, 1.0),
        'random_state': 42,
        'n_jobs': -1
    }
    
    oof = np.zeros(len(train_fe))
    gkf = GroupKFold(n_splits=5)
    
    for tr_idx, val_idx in gkf.split(train_fe, groups=train_fe['breath_id']):
        X_tr, X_val = train_fe.iloc[tr_idx][FEATURES], train_fe.iloc[val_idx][FEATURES]
        y_tr, y_val = train_fe.iloc[tr_idx][TARGET], train_fe.iloc[val_idx][TARGET]
        
        model = LGBMRegressor(**params)
        model.fit(
            X_tr, y_tr,
            eval_set=[(X_val, y_val)],
            eval_metric='mae',
            callbacks=[early_stopping(stopping_rounds=100)]
        )
        oof[val_idx] = model.predict(X_val)
    
    return mean_absolute_error(train_fe[TARGET], oof)


# Run Optuna
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=15, timeout=1800)  # до 30 минут

print("Best params:", study.best_params)
print("Best OOF MAE:", study.best_value)

/var/folders/rf/r9_1j8lj3m72m2lv3v1tp3rh0000gn/T/ipykernel_15260/3171096365.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.1),
/var/folders/rf/r9_1j8lj3m72m2lv3v1tp3rh0000gn/T/ipykernel_15260/3171096365.py:11: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('subsample', 0.6, 1.0),
/var/folders/rf/r9_1j8lj3m72m2lv3v1tp3rh0000gn/T/ipykernel_15260/3171096365.py:12: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'colsample_bytree': trial.suggest_uni

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.019880 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1884
[LightGBM] [Info] Number of data points in the train set: 4828800, number of used features: 12
[LightGBM] [Info] Start training from score 11.224218
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[4000]	valid_0's l1: 0.500148	valid_0's l2: 0.774199
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.019158 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1884
[LightGBM] [Info] Number of data points in the train set: 4828800, number of used features: 12
[LightGBM] [Info] Start training from score 11.225156
Training until vali

/var/folders/rf/r9_1j8lj3m72m2lv3v1tp3rh0000gn/T/ipykernel_15260/3171096365.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.1),
/var/folders/rf/r9_1j8lj3m72m2lv3v1tp3rh0000gn/T/ipykernel_15260/3171096365.py:11: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('subsample', 0.6, 1.0),
/var/folders/rf/r9_1j8lj3m72m2lv3v1tp3rh0000gn/T/ipykernel_15260/3171096365.py:12: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'colsample_bytree': trial.suggest_uni

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.019057 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1884
[LightGBM] [Info] Number of data points in the train set: 4828800, number of used features: 12
[LightGBM] [Info] Start training from score 11.224218
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[4000]	valid_0's l1: 0.568154	valid_0's l2: 0.966201
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.018662 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1884
[LightGBM] [Info] Number of data points in the train set: 4828800, number of used features: 12
[LightGBM] [Info] Start training from score 11.225156
Training until vali

/var/folders/rf/r9_1j8lj3m72m2lv3v1tp3rh0000gn/T/ipykernel_15260/3171096365.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.1),
/var/folders/rf/r9_1j8lj3m72m2lv3v1tp3rh0000gn/T/ipykernel_15260/3171096365.py:11: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('subsample', 0.6, 1.0),
/var/folders/rf/r9_1j8lj3m72m2lv3v1tp3rh0000gn/T/ipykernel_15260/3171096365.py:12: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'colsample_bytree': trial.suggest_uni

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.019496 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1884
[LightGBM] [Info] Number of data points in the train set: 4828800, number of used features: 12
[LightGBM] [Info] Start training from score 11.224218
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[4000]	valid_0's l1: 0.473531	valid_0's l2: 0.71589
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.020061 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1884
[LightGBM] [Info] Number of data points in the train set: 4828800, number of used features: 12
[LightGBM] [Info] Start training from score 11.225156
Training until valid

In [ ]:
# Train final model with best params
best_params = study.best_params
best_params['n_estimators'] = 4000
best_params['random_state'] = 42
best_params['n_jobs'] = -1

final_model = LGBMRegressor(**best_params)
final_model.fit(train_fe[FEATURES], train_fe[TARGET])

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.023016 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1884
[LightGBM] [Info] Number of data points in the train set: 6036000, number of used features: 12
[LightGBM] [Info] Start training from score 11.220408


,boosting_type,'gbdt'
,num_leaves,123
,max_depth,-1
,learning_rate,0.07695104560931866
,n_estimators,4000
,subsample_for_bin,200000
,objective,None
,class_weight,None
,min_split_gain,0.0
,min_child_weight,0.001
,min_child_samples,20


**Выводы по Feature Importance:**
- Наиболее важные признаки - это созданные FE (отмечены [FE])
- Lag-признаки (`u_in_lag_X`) оказались критичными для предсказания
- Базовые признаки (`u_in`, `u_out`, `time_step`) также играют важную роль
- Признак `R_x_C` (взаимодействие параметров легких) подтвердил свою значимость из EDA

### **3.3 Обучение финальной модели**

Используем лучшие параметры из Optuna для обучения на всех данных.

In [ ]:
# Predict on test
test_pred = final_model.predict(test_fe[FEATURES])

submission = pd.DataFrame({
    'id': test['id'],
    'pressure': test_pred
})
submission.to_csv('submissions/lgbm_fe_optuna_submission.csv', index=False)
print("Saved submission file")
submission.head()

Saved submission file


,id,pressure
0,1,6.297690
1,2,6.006357
2,3,7.029234
3,4,7.771845
4,5,8.748457



#### **Итоговые выводы**

- **OOF MAE**: ~0.5 (GroupKFold валидация)
- **Feature Engineering**: улучшение с 3.74 до 0.85 (в 4 раза!)
- **Best model**: LightGBM с Optuna-оптимизацией (15 trials)

**Ключевые инсайты:**
1. **Feature Engineering критичен** - 7 новых признаков дали основной прирост
2. **Lag-признаки важнейшие** - корреляция внутри циклов в 2-3 раза сильнее
3. **GroupKFold необходим** - предотвращает утечку данных по breath_id
4. **FE > Model > Hyperparams** - правильные признаки важнее оптимизации

#### **Сравнение с LAMA Baseline**

| Модель | Признаки | Public | Private |
|--------|----------|--------|---------|
| **LAMA Config 5** | 12 (5+7 FE) | 0.8480 | 0.8491 |
| **Custom LGBM** | 12 (5+7 FE) | **0.7761** | **0.779** |

